In [2]:
!pip install transformers

In [4]:
import pandas as pd

file_path = "data.xlsx"
sheet_name = "data"
column_name = "transcript"

data = pd.read_excel(file_path, sheet_name=sheet_name, usecols=["interviewees_name", column_name])
print(data)


                 interviewees_name  \
0    Inge-Juliana Sackmann Christy   
1                       Tom Foulds   
2                   Richard Eymann   
3                         Bob Cook   
4               George Warren Reed   
..                             ...   
589               Robert Ellingson   
590               Evelyn Ellingson   
591   Stirling Auchincloss Colgate   
592                  Colleen Black   
593                    Donald Ames   

                                            transcript  
0    : This is Wednesday, March 20. :    March 20. ...  
1    Okay. It is January 15th, 2019, and we are in ...  
2    Our second interview is with Richard Eymann, S...  
3    It’s April 29, 2019. Our first interview is wi...  
4    How would you describe your mother, grandmothe...  
..                                                 ...  
589  : My name is Robert Ellingson, and it’s spelle...  
590  : My name is Evelyn Ellingson, E-L-L-I-N-G-S-O...  
591  I’m Stirling Auchincloss 

In [5]:
name_data = pd.read_excel(file_path, sheet_name="profile_name", usecols=["name"])

In [6]:
first_row_text = name_data.iloc[0, 0]
print(first_row_text)

A. Becher


In [8]:
name_data

,name
0,A. Becher
1,A. Berkemeier
2,A. Bradley
3,A. Castle
4,A. Downey
...,...
14303,Zelmar Barson
14304,Zen B. Portwood
14305,Zenas J. Boone
14306,Zola DiMartino


In [9]:
import pandas as pd

# # 
# data = pd.DataFrame({"Name": ["A. Becher", "John Smith", "M. Johnson"]})

# # 
# data["Last Name"] = name_data["Name"].apply(lambda x: x.split(". ")[-1] if ". " in x else x)

last_name_data = name_data["name"].apply(lambda x: x.split(". ")[-1] if ". " in x else x)
#
print(last_name_data)


0                Becher
1            Berkemeier
2               Bradley
3                Castle
4                Downey
              ...      
14303     Zelmar Barson
14304          Portwood
14305             Boone
14306    Zola DiMartino
14307             Sojka
Name: name, Length: 14308, dtype: object


In [10]:
last_name_list = last_name_data.tolist()
full_name_list = name_data["name"].tolist()

In [11]:
print(len(last_name_list), len(full_name_list))

14308 14308


In [12]:
new_last_name_list = []

for name in last_name_list:
    split_names = name.split()
    new_last_name_list.extend(split_names)

In [13]:
all_names = new_last_name_list + full_name_list
print(len(all_names))

32839


In [14]:
from transformers import pipeline

# NER Model
ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")
interview_text = data.iloc[1, 1]

# Extract the name entity using the NER model
ner_results = ner_model(interview_text)

# A list of extracted names
extracted_names = [entity['word'] for entity in ner_results if entity['entity'] == 'I-PER']

print("Mentioned Name:")
name_res = []
for name in extracted_names:
    if name in all_names:
      name_res.append(name)
    else:
      continue

print(set(name_res))

Mentioned Name:
{'Steele', 'Karen', 'Tom', 'Frank'}


In [18]:
from transformers import pipeline

# Load the pre-trained NER model
ner_model = pipeline("ner", model="dbmdz/bert-large-cased-finetuned-conll03-english")

def run(i):
  interview_text = data.iloc[i, 1]
  interviewee = data.iloc[i, 0]
  ner_results = ner_model(interview_text)

  # 
  # extracted_names = [entity['word'] for entity in ner_results if entity['entity'] == 'I-PER']

  name_information = []

  names = []
  # 
  for name_entity in ner_results:
      if name_entity['entity'] == 'I-PER':
          name = name_entity['word']

          if name not in all_names:
            continue

          if len(name) < 3:
            continue

          start = name_entity['start']
          end = name_entity['end']
          
          # Look for sentences that contain people's names
          sentence_start = interview_text[:start].rfind('.') + 1
          sentence_end = interview_text[end:].find('.') + end + 1
          sentence = interview_text[sentence_start:sentence_end].strip()

          if name not in names:
            names.append(name)
            name_information.append((interviewee, name, start, end, sentence))

  return name_information

In [98]:
res = list()
for i in range(len(data)):
  name_list = run(i)
  res += name_list

df = pd.DataFrame(res, columns=['Interviewee', 'Name', 'Start', 'End', 'Sentence'])
df
file_path = "output.xlsx"
df.to_excel(file_path, index=False)

,Interviewee,Name,Start,End,Sentence
0,Inge-Juliana Sackmann Christy,Kelly,64,69,"I’m Cindy Kelly, and I’m in Pasadena, California."
1,Inge-Juliana Sackmann Christy,Christy,139,146,And I’m with Inge-Juliana Sackmann Christy.
2,Inge-Juliana Sackmann Christy,Sam,340,343,": I-n-g-e, Juliana, J-u-l-i-a-n-a, Sackmann, S..."
3,Tom Foulds,Tom,86,89,I am interviewing Tom Foulds.
4,Tom Foulds,Frank,169,174,"Tom T-o-m, Foulds F as in Frank, o-u-l-d-s."
...,...,...,...,...,...
2183,Stirling Auchincloss Colgate,Stirling,4,12,I’m Stirling Auchincloss Colgate.
2184,Colleen Black,Black,102,107,: We’re going to start with your name: could y...
2185,Donald Ames,Don,3,6,"So Don, why don’t you tell us your name and sp..."
2186,Donald Ames,Donald,65,71,"So Don, why don’t you tell us your name and sp..."


In [92]:
name_information

[('Inge-Juliana Sackmann Christy',
  'Kelly',
  64,
  69,
  'I’m Cindy Kelly, and I’m in Pasadena, California.'),
 ('Inge-Juliana Sackmann Christy',
  'Christy',
  139,
  146,
  'And I’m with Inge-Juliana Sackmann Christy.'),
 ('Inge-Juliana Sackmann Christy',
  'Sam',
  340,
  343,
  ': I-n-g-e, Juliana, J-u-l-i-a-n-a, Sackmann, S like Sam, a-c-k-m-a-n-n, Christy, C-h-r-i-s-t-y.')]

In [93]:
df1 = pd.DataFrame(name_information, columns=['Full Name', 'Name', 'Start', 'End', 'Sentence'])
print(df)

                       Full Name     Name  Start  End  \
0  Inge-Juliana Sackmann Christy    Kelly     64   69   
1  Inge-Juliana Sackmann Christy  Christy    139  146   
2  Inge-Juliana Sackmann Christy      Sam    340  343   

                                            Sentence  
0  I’m Cindy Kelly, and I’m in Pasadena, California.  
1        And I’m with Inge-Juliana Sackmann Christy.  
2  : I-n-g-e, Juliana, J-u-l-i-a-n-a, Sackmann, S...  


In [94]:
df1

,Full Name,Name,Start,End,Sentence
0,Inge-Juliana Sackmann Christy,Kelly,64,69,"I’m Cindy Kelly, and I’m in Pasadena, California."
1,Inge-Juliana Sackmann Christy,Christy,139,146,And I’m with Inge-Juliana Sackmann Christy.
2,Inge-Juliana Sackmann Christy,Sam,340,343,": I-n-g-e, Juliana, J-u-l-i-a-n-a, Sackmann, S..."


### Relation extract(It didn't work)

In [63]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

# Load the model and tokenizer
model_name = "Maklygin/mBert-relation-extraction-FT"
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForTokenClassification.from_pretrained(model_name, ignore_mismatched_sizes=True)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at Maklygin/mBert-relation-extraction-FT and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([6, 3072]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([6]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [67]:
def extract_name_information(text):
    tokens = tokenizer.tokenize(text)
    inputs = tokenizer.encode_plus(text, return_tensors="pt")
    outputs = model(**inputs)

    predicted_labels = outputs.logits.argmax(dim=2)[0]

    name_information = []
    current_name = ""
    current_information = ""

    for token, label_idx in zip(tokens, predicted_labels):
        label = model.config.id2label[label_idx.item()]  # Convert label_idx to Python scalars

        if label.startswith("B"):
            if current_name != "":
                name_information.append((current_name, current_information))
                current_name = ""
                current_information = ""

            current_name += token.replace("##", "")
        elif label.startswith("I"):
            current_information += token.replace("##", " ")
    
    if current_name != "":
        name_information.append((current_name, current_information))
    
    return name_information

In [68]:
text = "I’m Cindy Kelly. It is September 11, 2018, and I have with me John Fox. ..."
name_information = extract_name_information(text)

for name, information in name_information:
    print("Name:", name)
    print("Related info:", information)
    print()

In [69]:
name_information

[]

In [70]:
from transformers import pipeline

# Load a pre-trained relational extraction model
nlp = pipeline("ner", model="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract", tokenizer="microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract")

# Suppose your text data is in the variable text
text = "Steve Jobs was the CEO of Apple."

result = nlp(text)

# Prints the results of the relationship extraction
for relation in result:
    if relation['entity_group'] == 'PERSON':
        print(f"Person: {relation['word']}")
    else:
        print(f"Related Info: {relation['word']}")

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract were not used when initializing BertForTokenClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassific

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


KeyError: ignored